### TF-KOMPAS: Site caller

Updated 11/25/19

Author: Zachery Mielko

Optimization and review: Tiffany Ho

Review and feedback: Farica Zhuang

The script requires the following dependencies:

* pandas
* numpy
* biopython
* pybedtools

The script gives the following as **output**:
- Bed file of centered sites (Centered_KOMPAS.bed)
    - The Bed file is 0 based and of the format start inclusive, end exclusive. So half-open.
    - Columns: Chromosome, Start, End, Orientation, Threshold Score
- log file (optional)
- Diagnostic tsv file showing calls and which kpositions matched to which genomic coordinates (optional)

By default, the center position will be the midpoint of the core (rounded down), but it can be specifically chosen as an optional parameter.


**Threshold Score**

This is the threshold for when the site would be called. 

If k > core length (if multiple kmers exist that can fully describe the core), then the second score from **maximum** for those kmers is used. This is due to the requirnment of 2 overlapping kmers.

If k <= core length (if multiple (or 1) kmers are needed to describe the core fully), then the **minimum** score for those kmers is used because only when threshold is set to that minimum or below would the site be called

**Handles both palindromes and non-palindromes.**

If the query is for a palindrome, there is an option isPalindome to set to True. All it does is remove - strand calls, since any match on one strand is a match on another. If this is set to false for a palindrome, it should give + and - matches representing both centered positions, or 2 calls per binding site.

-------
Latest update (11/25/19)
* Diagnostic tsv file shows all searched peaks, even if no calls were made
* Threshold score is now set up so you can use it as a substitute for running at different thresholds. Represents practical threshold given 2 overlapping, not theoretical. 


Update (11/20/19):
* Fixed bug in coordinates for 0 vs 1 based. KOMPAS is now fully in-sync with half-open, 0 based formats used by bedtools, MACS, and UCSC (backend)


--------
To-do

1. Vectorize site calling
2. Introduce matrix output file of all called data as optional additional file for future use

In [2]:
################## Parameters ###############
# Files and folders
wkdir = "/Users/ZMielko/Desktop/Test_KOMPAS/"
peakFile = '/Users/ZMielko/Desktop/Test_KOMPAS/ets1_k562_ENCSR000BKQ_idr.txt'
genomeFile = '/Users/ZMielko/Desktop/In_Vivo_Project/Data/human_g1k_v37.fasta'
kmerFile = '/Users/ZMielko/Desktop/Test_KOMPAS/aligned_ets1_8mer.txt'
# kPosition of the core and where to center the call
# core is right exclusionary, left inclusive [)
core = (10,16) 
threshold = 0.3
isPalindrome = False
# Optional settings
saveDiagnostic = True # saves table with calling information for each seq
logFile = True
centerPos = 12

#############################################
if centerPos == 'default':
    centerPos = int((core[0] + core[1])/2)
##### Imports ####

import pandas as pd
import numpy as np
from Bio import SeqIO
from Bio.Seq import reverse_complement
import os
from itertools import groupby 
from operator import itemgetter
from pybedtools import BedTool

#### Functions ####
def readFasta(file):
    '''
    Reads a fasta file and turns it into a dataframe with forward and rc sequences
    '''
    entry = []
    with open(file, "r") as input_handle:
        for record in SeqIO.parse(input_handle, "fasta"):
            entry.append([record.id, str(record.seq).upper(), len(record.seq), str(reverse_complement(record.seq)).upper()])
    arr = np.array(entry)
    df = pd.DataFrame({'seq_id':arr[:, 0], 'fwd':arr[:,1], 'seq_len':arr[:,2], 'rev_comp':arr[:,3]})
    return df

def checkConsecutive(l): 
        return sorted(l) == l 

def group_by_tuple(a, b):
    '''
    a = [(1,2), (3,), (4, 5, 6)]
    b = [True, True, False, True, False, True]
    output = [(True, True), (False,), (True, False, True)]
    '''
    prev_tup_head, curr_tup_head = 0, 0
    grouped_b = []
    for tup in a:
        curr_tup_head = prev_tup_head+len(tup)
        boo = tuple(b[prev_tup_head:curr_tup_head])
        grouped_b.append(boo)
        prev_tup_head = curr_tup_head
    return grouped_b

def call_sites(match_pos, kPosition,kScore, k, coreLen, ReqKpos, orient,seqLen, ScoredKpos):
    '''
    Given matches and their kpositions within a site, returns called sites and their score
    '''
    groupedMpos, calledMpos, calledKpos, calledKCenter, calledKScore = [], [], [], [], []
    # get runs of consecutive numbers using groupby 
    for f, g in groupby(enumerate(match_pos), lambda x: x[1] - x[0]):
        groupedMpos.append(tuple(map(itemgetter(1), g)))
    groupedKpos = group_by_tuple(groupedMpos, kPosition)
    groupedKscore = group_by_tuple(groupedMpos, kScore)
    # If k is greater than core length
    if k > coreLen:
        for mpos_tuple, kpos_tuple, kscore_tuple in zip(groupedMpos, groupedKpos, groupedKscore):
            if len(mpos_tuple) >= 2 and checkConsecutive(list(kpos_tuple)) and len(ReqKpos & set(kpos_tuple)) > 0:
                # append tuple information for match position, kposition, center position
                calledMpos.append(mpos_tuple)
                calledKpos.append(kpos_tuple)
                centerk = (centerPos - kpos_tuple[0]) + mpos_tuple[0]
                calledKCenter.append(centerk)
                # Scoring, find all ReqKpos and their associated scores, get the max score
                site_score = []
                for kpos, score in zip(kpos_tuple, kscore_tuple):
                    if kpos in ScoredKpos:
                        site_score.append(score)
                site_score = sorted(site_score, reverse = True)[1] # Get the second highest score
                calledKScore.append(site_score)
    # if k is less than core length
    else:
        for mpos_tuple, kpos_tuple in zip(groupedMpos, groupedKpos, groupedKscore):
            if checkConsecutive(list(kpos_tuple)) and len(ReqKpos & set(kpos_tuple)) == len(ReqKpos):
                calledMpos.append(mpos_tuple)
                calledKpos.append(kpos_tuple)
                centerk = (centerPos - kpos_tuple[0]) + mpos_tuple[0] 
                calledKCenter.append(centerk)
                # Scoring, find all ReqKpos and their associated scores, get the min score
                site_score = 0.5 # maximum score
                for kpos, score in zip(kpos_tuple, kscore_tuple):
                    if kpos in ReqKpos and score < site_score:
                        site_score = score
                calledKScore.append(site_score)
    if len(calledKCenter) == 0:
        return([None,None,None, None])
    elif orient == 'fwd':
        return([calledKCenter,calledKScore, calledMpos, calledKpos])
    else:
        calledKCenter = list(map(lambda x: (seqLen - x) -1, calledKCenter)) 
        return([calledKCenter,calledKScore, calledMpos, calledKpos])
        
def parseID(df):
    """parseID takes the concatinated names given in fasta outputs from bedtool's getfasta 
    and turns them into bed compatible columns"""
    chrom, start, end = [], [], []
    for i in df.seq_id:
        cr = i.split(':')
        pos = cr[1].split('-')
        chrom.append(cr[0])
        start.append(int(pos[0]))
        end.append(int(pos[1]))
    df['Chromosome'] = chrom
    df['Start'] = start
    df['End'] = end
    return(df)

def convertToBed(df):
    chrom, start, orient,scores = [],[],[],[]
    for row in zip(df['Chromosome'],df['Start'],df['centerPlus'],df['scorePlus'],df['centerMinus'],df['scoreMinus']):
        if row[2]:
            for centerP, score in zip(row[2], row[3]): # + sites
                chrom.append(row[0])
                start.append(row[1] + centerP)
                orient.append('+')
                scores.append(score)
        if row[4]:
            for centerN, score in zip(row[4],row[5]): # - sites
                chrom.append(row[0])
                start.append(row[1] + centerN)
                orient.append('-')
                scores.append(score)
    bedDF = pd.DataFrame({'chrom':chrom,'start':start, 'end':start,'orient':orient, 'score':scores})
    bedDF['end'] = bedDF['end'] + 1 # exclusive end position
    bedDF = bedDF.drop_duplicates().sort_values(by=['chrom','start']) # some sites overlap, will call the same centers
    return(bedDF)
##### Read in kmer data and process ####
kmer = pd.read_csv(kmerFile, sep = '\t')
k = len(kmer['kmer'][0])
coreLen = core[1] - core[0]
# Find the kPositions required, any would be sufficient to call
if k > coreLen: 
    searchEnd = core[1]
    checkK = 0
    ReqKpos = set() #
    while checkK != core[0]:
        checkK = searchEnd - k
        if checkK <= core[0]:
            ReqKpos.add(checkK)
            searchEnd = searchEnd + 1
# Or find the group of all kPositions that are needed, all or none
else:
    searchStart = core[0]
    checkK = 0
    ReqKpos = set()
    while searchStart + k <= core[1]:
        ReqKpos.add(searchStart)
        searchStart = searchStart + 1
# Determine flanks of ReqKPos for threshold score reporting
ScoredKpos = ReqKpos.copy()
ScoredKpos.add(min(ReqKpos) - 1)
ScoredKpos.add(max(ReqKpos) + 1)        
        

kmerSearch = set(kmer[(kmer['Escore'] > threshold) & (kmer['kposition'].isin(ScoredKpos))]['kmer'])
kDict = dict(zip(kmer['kmer'],kmer['kposition']))
kScore = dict(zip(kmer['kmer'],kmer['Escore']))

##### Generate a fasta file from peakFile ####
peaks = pd.read_csv(peakFile, sep = '\t',header = None, usecols=[0,1,2])
# find out how long the peaks need to be to use them
if coreLen > k:
    minLen = coreLen
else:
    minLen = k + 1
peaks = peaks[peaks[2]-peaks[1] > (minLen)].drop_duplicates() # filter for short sequences the caller would have trouble with
peakBed = BedTool.from_dataframe(peaks)
peakSequence = peakBed.sequence(fi = genomeFile)
peakDF = readFasta(peakSequence.seqfn)
peakDF = parseID(peakDF)
del peakDF['seq_id']
peakDF[['seq_len', 'Start','End']] = peakDF[['seq_len', 'Start','End']].astype(int) 

def kmerMatch(seq,seqLen,orient, k, kmerSearch,kDict, coreLen, ReqKpos):
    """
    Returns matched positions in the sequence and their kpositions
    """
    pos, kpos,kscore = [], [], []
    for i in range(len(seq) - k + 1):
        window = seq[i:i+k]
        if window in kmerSearch:
            pos.append(i)
            kpos.append(kDict[window])
            kscore.append(kScore[window])
    if len(pos) <= 1: # must be at least 2 matches in the whole sequence, acts as a pre-filter
        return(pd.Series([None,None,None, None]))
    else:
        return(pd.Series(call_sites(pos,kpos,kscore,k, coreLen, ReqKpos, orient, seqLen,ScoredKpos))) 

# Run kmerMatch on the peaks
peakDF[["centerPlus","scorePlus","matchPlus","kpositionPlus"]] = peakDF.apply(lambda peakDF: kmerMatch(peakDF["fwd"],peakDF["seq_len"], 'fwd', k, kmerSearch, kDict, coreLen,ReqKpos), axis = 1) # search the forward strand
peakDF[["centerMinus","scoreMinus","matchMinus","kpositionMinus"]] = peakDF.apply(lambda peakDF: kmerMatch(peakDF["rev_comp"], peakDF["seq_len"], 'rc', k, kmerSearch, kDict,coreLen,ReqKpos), axis = 1) # search the reverse complement
if saveDiagnostic == True:
    peakDF[['Chromosome','Start','End','centerPlus','scorePlus','matchPlus','kpositionPlus','centerMinus','scoreMinus','matchMinus','kpositionMinus']].to_csv(f"{wkdir}Diagnostic_{threshold}_KOMPAS.tsv", sep = '\t')

peakDF = peakDF[~peakDF['centerPlus'].isna() | ~peakDF['centerMinus'].isna()]
finalBed = convertToBed(peakDF)
if isPalindrome == True:
    finalBed = finalBed.query("orient == '+'")
finalBed.to_csv(f'{wkdir}Centered_{threshold}_KOMPAS.bed', sep = '\t', header = None, index = False)



if logFile == True:
    ##################
    # Log the output #
    ##################
    f = open(wkdir + "/KOMPASLog.txt", "a")
    f.write("##### Parameters ##### \n")
    f.write(f"Peak file: {peakFile}"+ "\n")
    f.write(f"Genome file: {genomeFile}"+ "\n")
    f.write(f"kmer file: {kmerFile}"+ "\n")
    f.write(f"Core kPositions: {core}"+ "\n")
    f.write(f"Center kPositions: {centerPos}"+ "\n")
    f.write(f"Threshold: {threshold}"+ "\n")
    f.write("#Summary# \n")
    f.write(f"Final # of called sequences: {len(finalBed)}"+ "\n")
    f.close()